In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# this is for plotting datetime values in matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
df = pd.read_csv('../data/rnn_files/TimeSeriesData/Energy_Production.csv', index_col=0, parse_dates=True)
df.dropna(inplace=True)
print(len(df))
df.head()

In [ ]:
plt.figure(figsize=(16,4))
plt.title('Industrial protection for eletriciteis and gas utilities')
plt.ylabel('Index 2012=100, Not seasonally adjusted')
plt.grid(True)
plt.autoscale(axis='x', tight=True)
plt.plot(df['IPG2211A2N'])
plt.show()

In [ ]:
y = df['IPG2211A2N'].values.astype(float)

In [ ]:
window_size = 12
test_size = 12

train_set = y[:-window_size]
test_set = y[-window_size:]

print(f'Train: {len(train_set)}')
print(f'Test: {len(test_set)}')

In [ ]:
scalar = MinMaxScaler(feature_range=(-1,1))
train_norm = scalar.fit_transform(train_set.reshape(-1,1))

print(train_set[0])
print(train_norm[0])

In [ ]:
def prepare_seq_data(data, window_size):
    out = []
    for i in range(len(data)-window_size):
        window = data[i:i+window_size]
        label = data[i+window_size:i+window_size+1]     #this form is extendable by adding more than 1 to the window size
        out.append((window,label))
    return out

In [ ]:
train_norm = torch.FloatTensor(train_norm).view(-1)
train_data = prepare_seq_data(train_norm, window_size)
len(train_data)

In [ ]:
class LSTMNet(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, out_size=1):
        super().__init__()
        self.hidden_size = hidden_size
        
        # Add lstm layer
        self.lstm = nn.LSTM(input_size, hidden_size)
        
        # Add fully_connected layer
        self.linear = nn.Linear(hidden_size, out_size)
        
        # placeholder for hidden-state h and cell-state c
        self.hidden_state_cell_state = (torch.zeros(1,1,hidden_size), torch.zeros(1,1,hidden_size))
        
    def forward(self, data_seq):
        lstm_out, self.hidden_state_cell_state = self.lstm(data_seq.view(len(data_seq),1,-1), self.hidden_state_cell_state)
        pred = self.linear(lstm_out.view(len(data_seq),-1)[-1])
        return pred

In [ ]:
torch.manual_seed(101)
model = LSTMNet()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model

In [ ]:
import time
start_time = time.time()
epochs = 50

for i in range(epochs):
    for data_seq, y_train in train_data:
        optimizer.zero_grad()
        model.hidden_state_cell_state = (torch.zeros(1,1,model.hidden_size), torch.zeros(1,1,model.hidden_size))

        y_pred = model.forward(data_seq)
        
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()
        
    print(f'Epochs {i} loss:{loss}')

duration = time.time() - start_time
print(f'training time: {duration/60} mins')

In [ ]:
future = 12
preds = train_norm[-window_size:].tolist()

model.eval()
for i in range(future):
    seq = torch.FloatTensor(preds[-window_size:])
    with torch.no_grad():
        model.hidden_state_cell_state = (torch.zeros(1,1,model.hidden_size), torch.zeros(1,1,model.hidden_size))
        preds.append(model.forward(seq).item())

In [ ]:
preds[-window_size:]

In [ ]:
true_predictions = scalar.inverse_transform(np.array(preds[-window_size:]).reshape(-1,1))
true_predictions

In [ ]:
df.index

In [ ]:
x = np.arange('2018-02-01', '2019-02-01', dtype='datetime64[M]')
plt.figure(figsize=(16,4))
plt.title('Industrial protection for eletriciteis and gas utilities')
plt.ylabel('Index 2012=100, Not seasonally adjusted')
plt.grid(True)
plt.autoscale(axis='x', tight=True)
plt.plot(df['IPG2211A2N'])
plt.plot(x, true_predictions)
plt.show()

In [ ]:
x = np.arange('2018-02-01', '2019-02-01', dtype='datetime64[M]')
plt.figure(figsize=(16,4))
plt.title('Industrial protection for eletriciteis and gas utilities')
plt.ylabel('Index 2012=100, Not seasonally adjusted')
plt.grid(True)
plt.autoscale(axis='x', tight=True)
plt.plot(df['IPG2211A2N']['2017-01-01':])
plt.plot(x, true_predictions)
plt.show()